In [1]:
import findspark
findspark.init()
# import pyspark
# from pyspark.sql import SQLContext
# from pyspark import SparkContext    

# sc = SparkContext("local", "First App")
# sqlContext = SQLContext(sc)

In [2]:
import pyspark

In [3]:
from pyspark.sql import SQLContext
from pyspark import SparkContext  

In [ ]:
sc = SparkContext("local", "First App")
sqlContext = SQLContext(sc)

In [1]:
import sys
!{sys.executable} -m pip install xgboost

In [2]:
# pip install researchpy

In [3]:
pwd

'C:\\Users\\anuka\\Anaconda3\\Springboard_Capstone_Project_1'

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import random
import yaml
import seaborn as sns
sns.set()

from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import itertools as it

from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score

In [5]:
# Adds style to the printed dataframe output
from IPython.core.display import HTML
css = open('C:/Users/anuka/OneDrive/Desktop/Anu/Interview Prep/Springboard/Curriculum/Unit 5/Pandas from the ground up/pycon-pandas-tutorial-master/pycon-pandas-tutorial-master/style-table.css').read() + open('C:/Users/anuka/OneDrive/Desktop/Anu/Interview Prep/Springboard/Curriculum/Unit 5/Pandas from the ground up/pycon-pandas-tutorial-master/pycon-pandas-tutorial-master/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

## Load all datasets

In [2]:
os.chdir("C:/Users/anuka/OneDrive/Desktop/Anu/Interview Prep/Springboard/Curriculum/Capstone project 1/airbnb-recruiting-new-user-bookings")

# Load and clean training dataset
df_train = pd.read_csv("train_users_2.csv",parse_dates=['timestamp_first_active','date_account_created','date_first_booking'], skiprows=0, na_values = ['NaN'], encoding='utf-8')

# Load and clean test dataset
df_test = pd.read_csv("test_users.csv",parse_dates=['timestamp_first_active','date_account_created','date_first_booking'], skiprows=0, na_values = ['NaN'])

# Load and clean sessions dataset
df_sessions = pd.read_csv("sessions.csv",parse_dates=True, skiprows=0, na_values = ['NaN'])

# Load and clean countries dataset
df_countries = pd.read_csv("countries.csv",parse_dates=True, skiprows=0, na_values = ['NaN'])

# Load and clean age_gender_bkts dataset
# This dataset contains number of male/female in different age buckets who travelled to these destination countries in 2015 
df_age_gender_bkts = pd.read_csv("age_gender_bkts.csv",parse_dates=True, skiprows=0, na_values = ['NaN'])

# Get shape of all input datasets
print("The shape of training dataset is: ", df_train.shape)
print("The shape of test dataset is: ", df_test.shape)
print("The shape of sessions dataset is: ", df_sessions.shape)
print("The shape of countries dataset is: ", df_countries.shape)
print("The shape of age_gender_buckets dataset is: ", df_age_gender_bkts.shape)

The shape of training dataset is:  (213451, 16)
The shape of test dataset is:  (62096, 15)
The shape of sessions dataset is:  (10567737, 6)
The shape of countries dataset is:  (10, 7)
The shape of age_gender_buckets dataset is:  (420, 5)


In [3]:
df_test.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,2014-07-01 00:00:06,NaT,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,2014-07-01 00:00:51,NaT,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,2014-07-01 00:01:48,NaT,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,2014-07-01 00:02:15,NaT,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,2014-07-01 00:03:05,NaT,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


# Data Cleanup and Derived Features
1. Concatenate train and test data so that cleaning steps need to be done only once
2. Drop date_first_booking which is completely missing in the test data
3. Replace unknown values in gender and first_browser to NaN
4. Age will be filtered between 18-100 and set to NaN
5. Date time data of 'time_first_active' and 'timestamp_first_active' to be split into day, month and year columns
6. Drop date_account_created and timestamp_first_active columns
7. Remove leading and trailing spaces from language column

In [3]:
# Combining the test and train data. So that all the cleaning steps can be combined

test_ids = df_test['id']

df_train['type'] = 'train'
df_test['type'] = 'test'

df_all = pd.concat((df_train, df_test), axis = 0, ignore_index = True)

df_all = df_all.drop(['date_first_booking'], axis=1)

df_all.gender.replace('-unknown-', np.nan, inplace=True)
df_all.first_browser.replace('-unknown-', np.nan, inplace=True)

df_all.loc[df_all.age > 100, 'age'] = np.nan
df_all.loc[df_all.age < 18, 'age'] = np.nan

# Splitting date time data for date account created
df_all['dac_year'] = df_all.date_account_created.dt.year
df_all['dac_month'] = df_all.date_account_created.dt.month
df_all['dac_day'] = df_all.date_account_created.dt.day

# Splitting date time data for time first active
df_all['tfa_year'] = df_all.timestamp_first_active.dt.year
df_all['tfa_month'] = df_all.timestamp_first_active.dt.month
df_all['tfa_day'] = df_all.timestamp_first_active.dt.day

df_all.drop('date_account_created',1, inplace=True)
df_all.drop('timestamp_first_active',1, inplace=True)

df_all.language = df_all.language.str.strip()

print(df_all.shape)


# Aggregate sessions dataset at user_id level to merge with rest of the data
df_sessions_agg = df_sessions.groupby('user_id').agg({'action': 'count', 'action_type':'nunique','device_type':'nunique','secs_elapsed':np.mean}).reset_index()
df_sessions_agg.user_id = df_sessions_agg.user_id.str.strip()
df_sessions_agg.columns = ['user_id','cnt_action','cnt_uniq_action_type','cnt_uniq_dev_type','secs_per_session']
print(df_sessions.shape)
print(df_sessions_agg.shape)

df_all.id = df_all.id.str.strip()
df_all_1 = df_all.merge(df_sessions_agg, how = 'left', left_on=['id'],right_on = ['user_id'])
df_all_1.drop(['user_id'], axis = 1, inplace=True)
print(df_all_1.shape)

# Impute missing values
categorical = ['gender', 'first_affiliate_tracked', 'first_browser']
numeric = ['age','cnt_action','cnt_uniq_action_type','cnt_uniq_dev_type','secs_per_session']
impute_values = {}
for col in numeric:
    try:
        impute_values[col] = df_all_1[col].median()
    except:
         pass
for col in categorical:
    try:
        impute_values[col] = 'Missing'
    except:
        pass
df_all_1 = df_all_1.fillna(impute_values)
print(df_all_1.isnull().sum())


le = LabelEncoder()
df_all_1['country_dest_cd'] = le.fit_transform(df_all_1['country_destination'].astype(str))
selected_cols = ['gender', 'age', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 
                 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser', 'dac_year', 'dac_month', 
                 'dac_day', 'tfa_year', 'tfa_month', 'tfa_day', 'cnt_action', 'cnt_uniq_action_type', 'cnt_uniq_dev_type', 
                 'secs_per_session']
X = pd.get_dummies(df_all_1[df_all_1.type == 'train'][selected_cols])
y = df_all_1[df_all_1.type == 'train']['country_dest_cd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# sm = SMOTE(random_state=42)
# X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)
# y_train_bal.value_counts().plot.bar()

(275547, 20)
(10567737, 6)
(135483, 5)
(275547, 24)
id                             0
gender                         0
age                            0
signup_method                  0
signup_flow                    0
language                       0
affiliate_channel              0
affiliate_provider             0
first_affiliate_tracked        0
signup_app                     0
first_device_type              0
first_browser                  0
country_destination        62096
type                           0
dac_year                       0
dac_month                      0
dac_day                        0
tfa_year                       0
tfa_month                      0
tfa_day                        0
cnt_action                     0
cnt_uniq_action_type           0
cnt_uniq_dev_type              0
secs_per_session               0
dtype: int64


In [50]:
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate(y,y_hat,labels):
  print(classification_report(y,y_hat))
  cm = confusion_matrix(y,y_hat)
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
  cmat = pd.DataFrame(cm)
  cmat.columns = labels
  cmat.set_index([pd.Index(labels, '')],inplace=True)
  sns.heatmap(cmat,cmap="YlGnBu", annot=True)
  plt.title("Confusion Matrix")

pandas.core.frame.DataFrame

################################################################################################################

# 1. XGBoost Classifier

A great source to learn how Boosting works:
https://campus.datacamp.com/courses/extreme-gradient-boosting-with-xgboost/classification-with-xgboost?ex=8

XGBoost algorithm = Ensemble learning algorithm as it produces the output of many models for final prediction
1. Popular because of its speed and performance
2. Core algorithm is parallelizable
3. Consistently outperforms single algorithm method in ML competition
4. State of the art performance in many ML tasks - Mainly due to Boosting

Prediction in decision trees happens at the leaves of the tree

Boosting:
1. Not a specific machine learning algorithm
2. It's a meta algorithm that can be applied to a set of machine learning models
3. Ensemble meta algorithm used to convert many weak learners into a strong learner

Weak learner - ML algorithm that is slighly better than chance - decison tree whose predictions are slightly better than 50%
Strong learner - Any algorithm that can be tuned to achieve good performance

When to use XGBoost?
1. you have large number of training samples - greater than 1000 training samples and less than 100 features
2. Number of features < # training samples
3. You have a mix of categorical and numeric features or just numeric features

In [12]:
# Not run as was taking too long to run
# from sklearn.model_selection import GridSearchCV
# import numpy as np
# xgb = XGBClassifier(objective = 'multi:softprob', num_class = 12)
# hyper_params = {'n_estimators':[20,50], 'max_depth':[7], 'learning_rate': [0.3, 0.5], 'n_jobs':[10]} #, 'colsample_bynode': [1], }
# fitmodel = GridSearchCV(xgb, param_grid=hyper_params, cv=5, scoring="accuracy")
# fitmodel.fit(X_train_bal, y_train_bal)
# print(fitmodel.best_estimator_)
# print(fitmodel.best_params_)
# print(fitmodel.best_score_)
# print(accuracy_score(fitmodel.predict(X_test), y_test))

In [4]:
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=50, objective='multi:softprob')
xgb.fit(X_train, y_train)
y_train_pred = xgb.predict(X_train)
y_test_pred = xgb.predict(X_test)

print("Accuracy_score for train dataset", accuracy_score(y_train_pred, y_train))
print("Accuracy_score for test dataset", accuracy_score(y_test_pred, y_test))

Accuracy_score for train dataset 0.6542984305457953
Accuracy_score for test dataset 0.6422949590855144


In [10]:
len(X_train.columns)
len(X_test.columns)

143

In [12]:
len(selected_cols)

21

In [15]:
variables = X_train.columns.tolist()
X_hold = pd.get_dummies(df_all_1[df_all_1.type == 'test'][selected_cols]) # the hold out or test set (as per Kaggle) for predictions

hold_cols = set(X_hold.columns)
columns_not_present = set(variables) - hold_cols
for col in columns_not_present:
    X_hold[col] = 0
X_hold = X_hold[variables]
print('hold data size: ', X_test.shape[0])
        
y_hold_pred = xgb.predict_proba(X_hold) 

hold data size:  64036


In [7]:
categorical_all = ['gender', 'age', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider',
                   'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']

def de_one_hot_encode(s):
    for col in categorical_all:
        if(col in s):
            return col
    return s

# Get variable importance
variables = X_train.columns.tolist()
features = pd.DataFrame({'Importance':xgb.feature_importances_ , 'Variable':variables})
features['Variable'] = features['Variable'].apply(de_one_hot_encode)
features = features.groupby('Variable').sum().reset_index()
features = features.sort_values('Importance', ascending=False )
features = features[['Importance', 'Variable']]
display(features)
features.to_csv('feature_importances.csv')

,Importance,Variable
12,0.147144,gender
10,0.124529,first_browser
16,0.120520,signup_method
2,0.104899,age
9,0.080544,first_affiliate_tracked
0,0.080057,affiliate_channel
1,0.070542,affiliate_provider
19,0.062994,tfa_year
11,0.052968,first_device_type
4,0.050710,cnt_uniq_action_type


In [11]:
from sklearn.metrics import classification_report, confusion_matrix
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_test_pred))
print('\n')

print("=== Classification Report ===")
print(classification_report(y_test, y_test_pred))
print('\n')

=== Confusion Matrix ===
[[    0     0     0     0     0     0     0    88     0     0    83     0]
 [    0     0     0     0     0     0     0   217     0     0   203     0]
 [    0     0     0     0     0     0     0   181     0     0   153     0]
 [    0     0     0     0     0     0     0   375     0     0   311     0]
 [    0     0     0     0     0     0     0   819     0     0   686     0]
 [    0     0     0     0     0     0     0   405     0     0   343     0]
 [    0     0     0     0     0     0     0   487     0     0   379     0]
 [    0     0     1     0     0     0     1 32199     1     0  5000     4]
 [    0     0     0     0     0     0     0   109     0     0   116     0]
 [    0     0     0     0     0     0     0    42     0     1    34     0]
 [    0     0     1     0     0     0     2  9850     0     0  8929     3]
 [    0     0     0     0     0     0     0  1659     0     0  1353     1]]


=== Classification Report ===
              precision    recall  f1-scor

C:\Users\anuka\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
sub.head()

,id,country
0,5uwns89zht,NDF
1,5uwns89zht,US
2,5uwns89zht,nan
3,5uwns89zht,IT
4,5uwns89zht,FR


In [47]:
y_hold_pred[0:20]

array([[1.44933956e-03, 4.05452307e-03, 2.56952294e-03, 6.88130362e-03,
        6.93166116e-03, 5.35723567e-03, 8.07176065e-03, 6.85570240e-01,
        1.68390130e-03, 3.23357002e-04, 2.46408761e-01, 3.06983627e-02],
       [5.94785379e-04, 1.54551340e-03, 9.97917028e-04, 2.17778399e-03,
        5.09501947e-03, 2.87963334e-03, 3.15733487e-03, 8.92444134e-01,
        3.89892200e-04, 1.49503991e-04, 7.27813616e-02, 1.77871604e-02],
       [1.08308089e-03, 4.08504764e-03, 1.10268092e-03, 5.37870405e-03,
        1.04722921e-02, 5.59335621e-03, 7.38951052e-03, 8.29451084e-01,
        1.14177994e-03, 3.46786546e-04, 1.05490610e-01, 2.84650754e-02],
       [9.19978600e-04, 2.65046698e-03, 6.36610959e-04, 3.81609565e-03,
        6.32501021e-03, 3.48175457e-03, 4.85764444e-03, 9.12258446e-01,
        7.37682683e-04, 3.63039871e-04, 4.70006168e-02, 1.69526022e-02],
       [2.33743596e-03, 1.02233849e-02, 3.46002099e-03, 7.42188795e-03,
        3.68361846e-02, 1.27262296e-02, 1.65517554e-02, 1.40

In [46]:
# Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(test_ids)):
    idx = test_ids[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_hold_pred[i])[::-1])[:5].tolist()

# Generate submission
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.country = sub.country.str.replace('nan','NDF')
sub.to_csv('submission_29May_new.csv',index=False)

# 3. Multi-class Logistic Regression
Note that regularization is applied by default as per the links:
1. https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
2. https://stackoverflow.com/questions/31797149/l1-regularized-support-for-multinomial-logistic-regresion

In [30]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np
model = LogisticRegression(penalty = "l2", multi_class="multinomial", solver='lbfgs', max_iter=1000)
parameters = {"C": [0.1, 1, 10]}
fitmodel = GridSearchCV(model, param_grid=parameters, cv=5, scoring="accuracy")
fitmodel.fit(X_train, y_train.values.ravel())
print(fitmodel.best_estimator_)
print(fitmodel.best_params_)
print(fitmodel.best_score_)

print(accuracy_score(fitmodel.predict(X_test), y_test.values.ravel()))

C:\Users\anuka\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\anuka\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
{'C': 0.1}
0.5842117591941907
0.5806109063651695


C:\Users\anuka\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# 3. Random Forest Classifier

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier(random_state = 42, class_weight="balanced")
param_grid = {'n_estimators': [150], 'max_features': ['sqrt'], 'max_depth' : [12, 15], 'criterion' :['gini', 'entropy']}  # 'max_depth' : [4,5,6,7,8], 'criterion' :['gini', 'entropy']
# Random search of parameters
CV_rfc = GridSearchCV(estimator = rfc, param_grid = param_grid, cv = 5)
# Fit the model
CV_rfc.fit(X_train, y_train.values.ravel())
# print results
print(CV_rfc.best_params_)
print("Accuracy_score for train dataset", accuracy_score(CV_rfc.predict(X_train), y_train))
print("Accuracy_score for test dataset", accuracy_score(CV_rfc.predict(X_test), y_test))

{'criterion': 'entropy', 'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 150}
Accuracy_score for train dataset 0.6850985510156277
Accuracy_score for test dataset 0.49080204884752326
